In [1]:
from keras.layers import Input, Dense, Lambda, Flatten, Activation, Merge, Concatenate, Add
from keras import layers
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler

import numpy as np
from scipy.stats import sem
from scipy.spatial.distance import squareform, pdist
from keras import backend as K
from keras.models import load_model
import tensorflow as tf

from models import vgg
from models.cvae import CVAE
from models.cvae_mod import CVAE as CVAE_mod
from utils.losses import von_mises_log_likelihood_np
from utils.angles import deg2bit, bit2deg
from utils.losses import mad_loss_tf, cosine_loss_tf, von_mises_loss_tf, maad_from_deg
from utils.losses import gaussian_kl_divergence_tf, gaussian_kl_divergence_np
from utils.losses import gaussian_log_likelihood_np, gaussian_log_likelihood_scipy, gaussian_log_likelihood_tf
from utils.losses  import von_mises_log_likelihood_tf, von_mises_log_likelihood_np
from utils.sampling import sample_multiple_gauassians_np
from utils.towncentre import load_towncentre
from utils.experiements import get_experiment_id

Using TensorFlow backend.


In [2]:
xtr, ytr_deg, xval, yval_deg, xte, yte_deg = load_towncentre('data/TownCentre.pkl.gz', canonical_split=True, verbose=1)

image_height, image_width = xtr.shape[1], xtr.shape[2]
ytr_bit = deg2bit(ytr_deg)
yval_bit = deg2bit(yval_deg)
yte_bit = deg2bit(yte_deg)
yte_rad = np.deg2rad(yte_deg)

image_height, image_width, n_channels = xtr.shape[1:]
flatten_x_shape = xtr[0].flatten().shape[0]
phi_shape = yte_bit.shape[1]

Number of train samples: 6916
Number of validation samples: 874
Number of test samples: 904


In [3]:
n_u = 8
cvae = CVAE(n_hidden_units=n_u)
cvae_ckpt_path = 'logs/best_models/cvae/3/cvae.full_model.trial_2.best.weights.hdf5'
cvae.full_model.load_weights(cvae_ckpt_path)

In [4]:
results_cvae = dict()
#results_cvae['train'] = cvaekl.evaluate(xtr, ytr_deg, 'train')
results_cvae['validation'] = cvae.evaluate(xval, yval_deg, 'validation')
results_cvae['test'] = cvae.evaluate(xte, yte_deg, 'test')

MAAD error (validation) : 26.917997 ± 1.178502SEM
ELBO (validation) : -0.700545 ± 0.049847SEM
KL-div (validation) : 0.350774 ± 0.019146SEM
MAAD error (test) : 27.201755 ± 1.218712SEM
ELBO (test) : -0.717721 ± 0.053369SEM
KL-div (test) : 0.322192 ± 0.018516SEM


Gaussian log-likelihood

$\log L(x)= -\frac{1}{2} \left( \log (|\boldsymbol\Sigma|\,) + (\mathbf{x}-\boldsymbol\mu)^{\rm T}\boldsymbol\Sigma^{-1}(\mathbf{x}-\boldsymbol\mu) + n\log(2\pi) \right)$

In case $\boldsymbol\Sigma = diag(\sigma_1^2, \dots, \sigma_n^2)$

$\log L(x)= -\frac{1}{2} \left( (\sum_{i=1}^{n}{\log \sigma^2_i}) + \sum_{i=1}^{n}{\frac{(x_i-\mu_i)^2}{\sigma^2_i}} + n\log(2\pi) \right)$


In [5]:
def get_model_samples(cvae, x, y, n_samples=10):
    
    n_points = x.shape[0]
    
    x_tiled = np.repeat(x_ex, repeats=n_samples, axis=0)
    y_tiled = np.repeat(y_ex, repeats=n_samples, axis=0)
    
    output = cvae.get_full_output(x, y)
    mu_prior = output['mu_prior']
    std_prior = np.exp(output['log_sigma_prior'] / 2)
    mu_encoder = output['mu_encoder']
    std_encoder = np.exp(output['log_sigma_encoder'] / 2)
    u_encoder = output['u_encoder']
    mu_pred = output['mu_pred']
    
    u_samples = np.zeros([n_points, n_samples, cvae.n_u])
    
    mu_preds = np.zeros([n_points, n_samples, 2])
    kappa_preds = np.zeros([n_points, n_samples, 1])
        
    for sid in range(0, n_samples):
        
        kappa_preds[:, sid, :] = output['kappa_pred']
        
        #vm_likelihood[:, sid] = np.squeeze(np.exp(von_mises_log_likelihood_np(y, mu_pred, kappa_pred)))
        u_samples[:, sid, :] = u_encoder
    
    prior_log_likelihood =  gaussian_log_likelihood_np(mu_prior, std_prior, u_samples)
    encoder_log_likelihood = gaussian_log_likelihood_np(mu_encoder, std_encoder, u_samples)
    
    sample_weight = np.exp(prior_log_likelihood - encoder_log_likelihood)
    
    importance_likelihoods = np.log(np.mean(vm_likelihood*sample_weight, axis=1))
    
    return u_samples, mu_pred

#### Improtance sampling
$ p(\phi| x) \approx \frac{1}{S} \sum_{i=1}^{S}{\frac{ p(\phi | x, u_i) p(u_i|x)  }{ q(u_i| x, \phi)}}$

where

$ p(u|x) \sim \mathcal{N}(\mu_1(x), \sigma_1(x)) $

$ q(u|x,\phi) \sim \mathcal{N}(\mu_2(x, \phi), \sigma_2(x, \phi)) $

$ p(\phi|x, u) \sim \mathcal{VM}(\mu(x,u), \kappa(x,u)) $


In [3]:
n_u = 8
cvae_ckpt_path = 'logs/best_models/cvae/3/cvae.full_model.trial_2.best.weights.hdf5'

cvae = CVAE_mod(n_hidden_units=n_u, n_samples=10)
#cvae.full_model.load_weights(cvae_ckpt_path)

In [4]:
x_ex = xtr[0:10]
y_ex = ytr_bit[0:10]

In [ ]:
cvae.full_model.fit([xtr, ytr_bit], [ytr_bit], batch_size=10, epochs=50)

Epoch 1/50
6916/6916 [==============================] - 195s - loss: 2.0675   
Epoch 2/50
6916/6916 [==============================] - 184s - loss: 1.8160   
Epoch 3/50
5740/6916 [=======================>......] - ETA: 30s - loss: 1.8030

In [21]:
out = cvae.full_model.predict([x_ex, y_ex])
y = y_ex

In [25]:
out_ph = tf.placeholder(dtype=tf.float32, shape=[None, 152])
y_ph = tf.placeholder(dtype=tf.float32, shape=[None, 2])

impll_tf = cvae.importance_log_likelihood_tf(y_ph, out_ph)

In [26]:
with tf.Session() as sess:
    ll = impll_tf.eval(feed_dict={out_ph:out, y_ph:y})

In [27]:
ll

1.8896326